In [1]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-1.2.jar 

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-1.2.jar
Finished download of spark-kernel-brunel-all-1.2.jar


In [4]:
%AddDeps com.databricks spark-csv_2.10 1.3.0 --transitive

Marking com.databricks:spark-csv_2.10:1.3.0 for download
Preparing to fetch from:
-> file:/var/folders/qy/z43k21xd5v9db55bkb41svmw0000gn/T/toree_add_deps7295586412712904848/
-> https://repo1.maven.org/maven2
-> New file at /var/folders/qy/z43k21xd5v9db55bkb41svmw0000gn/T/toree_add_deps7295586412712904848/https/repo1.maven.org/maven2/org/apache/commons/commons-csv/1.1/commons-csv-1.1.jar
-> New file at /var/folders/qy/z43k21xd5v9db55bkb41svmw0000gn/T/toree_add_deps7295586412712904848/https/repo1.maven.org/maven2/com/databricks/spark-csv_2.10/1.3.0/spark-csv_2.10-1.3.0.jar
-> New file at /var/folders/qy/z43k21xd5v9db55bkb41svmw0000gn/T/toree_add_deps7295586412712904848/https/repo1.maven.org/maven2/com/univocity/univocity-parsers/1.5.1/univocity-parsers-1.5.1.jar


In [6]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._ 

val sqlContext = new SQLContext(sc)

//Data is not aggregated
val co2 = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load("Co2all.csv")

//Add a column that has Year as a true date type
val convertDate = udf {
    (year:Int) => year+ "-01-01 " +year + " 00:00 UTC"
}
val co2dates = co2.withColumn("year_as_date", to_date(convertDate(co2("Year"))) )

//Data is aggregated by Spark
val co2agg = co2.groupBy("CO2 per capita").agg( avg("value") as "Mean Co2")

In [7]:
%%brunel data('co2agg') map(low) x(CO2_per_capita) color(Mean_Co2) tooltip(#all):: width=800, height=500